В наличии данные по карточным транзакциям клиентов за 3 месяца и файл с дополнительными сведениями о клиентах.


**Задача:**


Разделить клиентов на 2 равномерные группы для проведения аб-эксперимента, цель которого – увеличение транзакционной активности.


# **0. Предобработка данных**

Импортируем нужные библиотеки:

In [ ]:
import pandas as pd
import datetime
import numpy as np
from datetime import date
import plotly.express as px

In [ ]:
# чтобы не получать лишние предупреждения
pd.options.mode.chained_assignment = None

Файл с данными лежит в google drive.

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
df = pd.read_csv('/gdrive/MyDrive/clients_small.csv', sep=';')

In [ ]:
df.head(1)

,customer_id,card_id,region_name,sex,birth_date
0,02F4DB34C5696A668068CA3C1DE7C2D5,6600B4FDA036A6F00D464541EAD3B21F,Москва,F,1938-08-18


Добавим столбец возраста в исходный датафрейм.

Для этого переведем стоблец birth_date в формат datetime

In [ ]:
df['birth_date'] = pd.to_datetime(df['birth_date'], errors='coerce')

Введем функцию рассчитывающую возраст человека:

In [ ]:
def calculate_age(born):
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

Таким образом получим столбец с возрастом.

In [ ]:
df['age'] = [calculate_age(df.loc[row,'birth_date'])
            for row in range(len(df))]

# **1. Проверим, что в имеющихся данных каждому пользователю присвоен один пол и один регион.**

Чтоб проверить наличие одного пола у одного кастомера, создадим сводную таблицу и для каждого уникального customer_id выведем какой пол ему соответствует.

In [ ]:
table_sex_all = df.pivot_table(index='customer_id', columns='sex', 
                           values='region_name', aggfunc='count').reset_index()
  

In [ ]:
table_sex_all.head(1)

sex,customer_id,F,M
0,00018A6EE50848556CE7B7C920C2776F,NaN,4.0


Как известно при сложении NaN с числом, мы получаем на выходе NaN. Это значит что, если у каждого кастомера один пол, то на выходе мы получим все NaN для столбца sum.

In [ ]:
table_sex_all['sum'] = table_sex_all['F']+table_sex_all['M']

In [ ]:
table_sex_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6875 entries, 0 to 6874
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   customer_id  6875 non-null   object 
 1   F            3387 non-null   float64
 2   M            3488 non-null   float64
 3   sum          0 non-null      float64
dtypes: float64(3), object(1)
memory usage: 215.0+ KB


Как мы видим в столбце sum все значения NaN. Значит у кастомеров пол только 1.

Чтоб проверить регионы для каждого пользователя мы также построим сводную таблицу.

In [ ]:
table_all_reg = df.pivot_table(index='customer_id', values='region_name', 
                          aggfunc='nunique').reset_index()
  

In [ ]:
table_all_reg[table_all_reg['region_name']>1]

,customer_id,region_name


Аналогично прошлому, видим что нет ни одного пользователя, у которого указано два или более регионов.

# **2. Равномерное разделение пользователей на две группы.**

Для начала создадим список уникальных значений customer_id, чтоб по нему разделить исходный датафрейм.

In [ ]:
frame_id = pd.Series(df['customer_id'].unique())

Разделим пополам список customer_id и для каждого из них получим датафреймы.

In [ ]:
group_1 = frame_id.sample(frac=0.5)
group_2 = frame_id.loc[~frame_id.index.isin(group_1.index)]

In [ ]:
result_group_1 = df[df['customer_id'].isin(list(group_1))]
result_group_2 = df[df['customer_id'].isin(list(group_2))]

In [ ]:
result_group_1 = result_group_1.reset_index(drop=True)
result_group_2 = result_group_2.reset_index(drop=True)

Таким образом, у нас есть две группы пользователей из имеющегося датафрейма.

1. result_group_1
2. result_group_2

# **3. Анализ полученных групп**

Для удобства разделения данных добавим в стартовый датафрейм столбец разделения по группам.

In [ ]:
df['group'] = [1
               if df.loc[row, 'customer_id'] in (list(group_2))
               else 2
               for row in range(len(df))]

Для получения статистики по полу, возрасту и регионам нам надо оставить в полученных группах только уникальные значения customer_id, для этого дропнем дубликаты по этому столбцу.

In [ ]:
df_id = df.drop_duplicates(subset=['customer_id'])

# **3.1 Рассмотрим какой процент женских и мужских клиентов в каждой группе:**

In [ ]:
table_sex = df_id.pivot_table(index=['sex', 'group'], values='customer_id', 
                              aggfunc='count').reset_index()

In [ ]:
table_sex

,sex,group,customer_id
0,F,1,1719
1,F,2,1668
2,M,1,1719
3,M,2,1769


Добавим столбец, показывающий процент каждого типа данных относительно всего количества. Для этого напишем функцию:

In [ ]:
def percent(table):
  table['sum'] = [table[table['group'] == 1]['customer_id'].sum()
                    if table.loc[row,'group'] == 1
                    else table[table['group'] == 2]['customer_id'].sum()
                    for row in range(len(table))]
  table['percent'] = round(table['customer_id']/ table['sum'] *100,2) 
  del table['sum']
  return table        

Таким образом мы получаем такую сводную таблицу:

In [ ]:
percent(table_sex)

,sex,group,customer_id,percent
0,F,1,1719,50.00
1,F,2,1668,48.53
2,M,1,1719,50.00
3,M,2,1769,51.47


Как видно из таблицы, в первой группе количество женщин и мужчин в группе одинаково, во второй группе 51.47% мужчин и 48.53% женщин.

# **3.2 Рассмотрим какое распределение клиентов по регионам в каждой группе:**

In [ ]:
table_region = df_id.pivot_table(index=['region_name', 'group'], values='customer_id', 
                              aggfunc='count').reset_index()

Добавим процент людей из каждого региона от общего числа

In [ ]:
table_region = percent(table_region)
table_region

,region_name,group,customer_id,percent
0,Алтайский край,1,21,0.61
1,Алтайский край,2,19,0.55
2,Амурская область,1,48,1.41
3,Амурская область,2,53,1.54
4,Архангельская область,1,17,0.50
...,...,...,...,...
131,Челябинская область,2,65,1.89
132,Чувашская Республика,1,8,0.23
133,Чувашская Республика,2,6,0.17
134,Ярославская область,1,21,0.61


In [ ]:
px.bar(table_region, x='region_name', y='percent', color='group')

Клиенты достаточно равномерно распределены по регионам для каждой из представленных групп.

# **3.3 Рассмотрим какое распределение клиентов по возрастам в каждой группе:**

Чтоб построить распределение клиентов по возрастам, добавим шкалу возрастов. Для этого разделим имеющиеся возраста на кластеры по 10 лет.

In [ ]:
for i in range(20, 100, 10):
  temp_data = df_id[(df_id['age'] >= i - 10) & (df_id['age'] <= i)]
  temp_index = list(temp_data.index)
  df_id.loc[temp_index, 'age_group'] = str(i - 10) + ' - ' + str(i)

Точно также как и раньше построим сводную таблицу данных и добавим в нее процент числа клиентов относительно всех.

In [ ]:
table_age = df_id.pivot_table(index=['age_group', 'group'], values='customer_id', 
                              aggfunc='count').reset_index()

In [ ]:
table_age = percent(table_age)
table_age

,age_group,group,customer_id,percent
0,10 - 20,1,19,0.55
1,10 - 20,2,13,0.37
2,20 - 30,1,552,15.83
3,20 - 30,2,511,14.66
4,30 - 40,1,1185,33.99
5,30 - 40,2,1182,33.91
6,40 - 50,1,943,27.05
7,40 - 50,2,937,26.88
8,50 - 60,1,533,15.29
9,50 - 60,2,553,15.86


In [ ]:
px.bar(table_age, x='age_group', y='percent', color='group')


Таким образом видно, что полученные группы клиентов достаточно равномерно распределены по возрастам.